In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import scrapy
import random
import time
from urllib.parse import urljoin
import re
from tqdm import tqdm
import chardet

## 1º Jornal Online - Agência Brasil

In [ ]:
def get_url_from_agencia_brasil(index):
	

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		for i in range(1, 50):
				
			search_url = f"https://busca.ebc.com.br/nodes?page={i}utf8=%E2%9C%93&q={query}&site_id=agenciabrasil"
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			#print(links)
			lista.extend(links)
		# Break se não achar mais links (opcional)
			if not links:
				break

		return lista
	
	def encontra_links_do_html(soup):
		anchors = soup.find_all("a")
		links_encontrados = set()  # usando set para evitar duplicatas

		for tag in anchors:
			href = tag.get("href")
			if href and href.startswith("http://agenciabrasil"):
				links_encontrados.add(href.strip())

		return list(links_encontrados)
			
	return main(index)

def pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave):
	fim = dict()
	for chave in palavras_chave:
		url = get_url_from_agencia_brasil(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	return fim

palavras_chave = ['presidente', 'China', 'eua', 'Trump']

a = pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave)
a

['http://agenciabrasil.ebc.com.br/direitos-humanos/noticia/2025-04/fruto-da-mobilizacao-indigena-apib-completa-20-anos-de-existencia', 'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-4', 'http://agenciabrasil.ebc.com.br/internacional/noticia/2025-04/atentado-bomba-na-colombia-mata-duas-pessoas-e-deixa-24-feridas', 'http://agenciabrasil.ebc.com.br/es/politica/noticia/2025-04/lula-viajara-china-en-mayo-para-nuevo-encuentro-con-xi-jinping', 'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-5', 'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-6', 'http://agenciabrasil.ebc.com.br/en/politica/noticia/2025-04/lula-visit-china-may-another-meeting-xi-jinping', 'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-3', 'http://agenciabrasil.ebc.com.br/geral/noticia/2025-04/feriadao-deve-ter-chuva-e-qu

Exception ignored in: <function tqdm.__del__ at 0x0000022CF4D0A020>
Traceback (most recent call last):
  File "c:\Users\T-Gamer\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\T-Gamer\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\std.py", line 1277, in close
    if self.last_print_t < self.start_t + self.delay:
       ^^^^^^^^^^^^^^^^^
AttributeError: 'tqdm' object has no attribute 'last_print_t'


['http://agenciabrasil.ebc.com.br/foto/2025-04/08042025-chegada-tegucigalpa-honduras-1744198995', 'http://agenciabrasil.ebc.com.br/foto/2025-04/censo-escolar-2024-1744213540', 'http://agenciabrasil.ebc.com.br/internacional/noticia/2025-04/acoes-da-china-sobem-apos-pais-indicar-abertura-para-negociar-tarifas', 'http://agenciabrasil.ebc.com.br/justica/noticia/2025-04/agu-pede-investigacao-de-deputado-por-declaracoes-contra-lula', 'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/brics-lanca-concurso-para-startups-lideradas-por-mulheres', 'http://agenciabrasil.ebc.com.br/foto/2025-04/censo-escolar-2024-1744213540-5', 'http://agenciabrasil.ebc.com.br/foto/2025-04/08042025-chegada-tegucigalpa-honduras-1744198993', 'http://agenciabrasil.ebc.com.br/justica/noticia/2025-04/tre-sp-cassa-mandatos-do-prefeito-e-da-vice-prefeita-de-barueri', 'http://agenciabrasil.ebc.com.br/foto/2025-04/ministro-padilha-no-atl-1744218818-3', 'http://agenciabrasil.ebc.com.br/justica/noticia/2025-04/relator-

{'presidente': ['http://agenciabrasil.ebc.com.br/direitos-humanos/noticia/2025-04/fruto-da-mobilizacao-indigena-apib-completa-20-anos-de-existencia',
  'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-4',
  'http://agenciabrasil.ebc.com.br/internacional/noticia/2025-04/atentado-bomba-na-colombia-mata-duas-pessoas-e-deixa-24-feridas',
  'http://agenciabrasil.ebc.com.br/es/politica/noticia/2025-04/lula-viajara-china-en-mayo-para-nuevo-encuentro-con-xi-jinping',
  'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-5',
  'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-6',
  'http://agenciabrasil.ebc.com.br/en/politica/noticia/2025-04/lula-visit-china-may-another-meeting-xi-jinping',
  'http://agenciabrasil.ebc.com.br/foto/2025-04/terra-indigena-erikpatsa-do-povo-rikbaktsa-1745001084-3',
  'http://agenciabrasil.ebc.com.br/geral/noticia/2025-

In [81]:
list(a.keys())

['presidente', 'China', 'eua', 'Trump']

In [ ]:
list(a.values())


IndexError: list index out of range

In [68]:
def extrai_dados_noticia(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
        "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
    ]

    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "pt-BR,pt;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")

    # Título
    titulo_tag = soup.find("h1")
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Título não encontrado"

    # Data de publicação atualizada
    data_tag = soup.find("div", class_="data")
    if data_tag:
        data_texto = data_tag.get_text(strip=True)
        data_publicacao = data_texto.replace("Publicado em", "").strip()
    else:
        data_publicacao = "Data não encontrada"

    # Conteúdo principal atualizado
    conteudo_tag = soup.find("div", class_="conteudo-noticia")
    if conteudo_tag:
        paragrafos = conteudo_tag.find_all("p")
        conteudo = "\n".join(p.get_text(strip=True) for p in paragrafos if p.get_text(strip=True))
    else:
        conteudo = "Conteúdo não encontrado"

    return {
        "titulo": titulo,
        "data_publicacao": data_publicacao,
        "conteudo": conteudo
    }



In [ ]:
extrai_dados_noticia("https://agenciabrasil.ebc.com.br/direitos-humanos/noticia/2025-04/fruto-da-mobilizacao-indigena-apib-completa-20-anos-de-existencia")

In [ ]:
### Estrutura os dados em dicionários ###
b_dicionario = dict()
for b in list(a.keys()):
    c_dicionario = dict()
    for c in list(a.values())[0]:
        c_dicionario[c] = extrai_dados_noticia(c)
    b_dicionario[b] = c_dicionario


In [103]:
# Transformar b_dicionario em uma lista de linhas para o DataFrame
linhas = []
for chave_principal, sub_dicionario in b_dicionario.items():
    for url, dados in sub_dicionario.items():
        linha = {
            "Palavra-chave": chave_principal,
            "URL": url,
            "Título": dados.get("titulo", "Título não encontrado"),
            "Data de Publicação": dados.get("data_publicacao", "Data não encontrada"),
            "Conteúdo": dados.get("conteudo", "Conteúdo não encontrado"),
        }
        linhas.append(linha)

# Criar o DataFrame
df = pd.DataFrame(linhas)

# Exibir o DataFrame
print(df)

     Palavra-chave                                                URL  \
0       presidente  http://agenciabrasil.ebc.com.br/direitos-human...   
1       presidente  http://agenciabrasil.ebc.com.br/foto/2025-04/t...   
2       presidente  http://agenciabrasil.ebc.com.br/internacional/...   
3       presidente  http://agenciabrasil.ebc.com.br/es/politica/no...   
4       presidente  http://agenciabrasil.ebc.com.br/foto/2025-04/t...   
...            ...                                                ...   
2935         Trump  http://agenciabrasil.ebc.com.br/politica/notic...   
2936         Trump  http://agenciabrasil.ebc.com.br/politica/notic...   
2937         Trump  http://agenciabrasil.ebc.com.br/justica/notici...   
2938         Trump  http://agenciabrasil.ebc.com.br/internacional/...   
2939         Trump  http://agenciabrasil.ebc.com.br/justica/notici...   

                                                 Título  \
0     Fruto da mobilização indígena, Apib completa 2...   
1    

In [ ]:
### Impostação para o Excel ###

import openpyxl
#df.to_excel("noticias_agencia_brasil.xlsx", index=False, engine='openpyxl')

## 2º Jornal Online - Brasil de Fato

In [11]:
import chardet

def get_url_from_Brasil_de_fato(index):
	
	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		for i in range(1, 50):
				
			search_url = f"https://www.brasildefato.com.br/page/{i}/?s={query}"
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			lista.extend(links)
		# Break se não achar mais links (opcional)
			if not links:
				break

		return lista
	
	def encontra_links_do_html(soup):
		anchors = soup.find_all("a")
		links_encontrados = set()  # usando set para evitar duplicatas
		for tag in anchors:
			href = tag.get("href")
			if href and href.startswith("https://www.brasildefato"):
				links_encontrados.add(href.strip())

		return list(links_encontrados)
			
	return main(index)

def pega_url_de_lista_com_timesleep_Brasil_de_fato(palavras_chave):
	fim = dict()
	for chave in palavras_chave:
		url = get_url_from_Brasil_de_fato(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	return fim

palavras_chave = ['China']

a = pega_url_de_lista_com_timesleep_Brasil_de_fato(palavras_chave)
a

{'China': ['https://www.brasildefato.com.br/editoria/english/struggles/?v=2',
  'https://www.brasildefato.com.br/editoria/direitos/',
  'https://www.brasildefato.com.br/editoria/politica/eleicoes/',
  'https://www.brasildefato.com.br/editoria/educacao/',
  'https://www.brasildefato.com.br/editoria/internacional/',
  'https://www.brasildefato.com.br/editoria/editorial/',
  'https://www.brasildefato.com.br/editoria/transporte/',
  'https://www.brasildefato.com.br/editoria/bem-viver/',
  'https://www.brasildefato.com.br/english/',
  'https://www.brasildefato.com.br/pb/',
  'https://www.brasildefato.com.br/2025/04/11/china-reage-e-aumenta-tarifas-sobre-produtos-dos-eua-para-125/',
  'https://www.brasildefato.com.br/editoria/bem-viver/cultura/',
  'https://www.brasildefato.com.br/ce/',
  'https://www.brasildefato.com.br/2025/04/15/em-viagem-de-xi-jinping-china-fortalece-acordos-economicos-na-asia-em-guerra-comercial-com-eua/',
  'https://www.brasildefato.com.br/page/388/?s=China',
  'https:

## 3º Jornal Online - NPR (EUA)

In [36]:


def get_url_from_the_guardian(index):
	
	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		for i in range(1, 50):
				
			search_url = f"https://www.google.co.uk/search?q={query}&as_sitesearch=www.theguardian.com&sei=bDYEaIibCYeI4dUP46aNgAk"
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			#print(soup)
			links = encontra_links_do_html(soup)
			#print(links)
			lista.extend(links)
		# Break se não achar mais links (opcional)
			if not links:
				break

		return lista
	
	def encontra_links_do_html(soup):
		anchors = soup.find_all("a")
		links_encontrados = set()  # usando set para evitar duplicatas
		print(anchors)
		for tag in anchors:
			href = tag.get("href")
			if href and href.startswith("https://www.theguardian.com"):
				links_encontrados.add(href.strip())

		return list(links_encontrados)
			
	return main(index)

def pega_url_de_lista_com_timesleep_the_guardian(palavras_chave):
	fim = dict()
	for chave in palavras_chave:
		url = get_url_from_the_guardian(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	return fim

palavras_chave = ['China and Trump']

a = pega_url_de_lista_com_timesleep_the_guardian(palavras_chave)
a

[<a href="/httpservice/retry/enablejs?sei=5jcEaMCcBqvV1sQP7euRkQ8">aqui</a>, <a href="/search?q=China+and+Trump+site:www.theguardian.com&amp;sca_esv=9bc03c031fd543e7&amp;emsg=SG_REL&amp;sei=5jcEaMCcBqvV1sQP7euRkQ8">click here</a>, <a href="https://support.google.com/websearch">feedback</a>]


{'China and Trump': []}

In [32]:
list(a.values())[0][70:90]

['https://www.npr.org/podcasts/510374/trumps-terms',
 'https://www.npr.org/sections/music-features',
 'https://www.npr.org/series/tiny-desk-concerts/',
 'https://www.npr.org/sections/allsongs/606254804/new-music-friday',
 'https://www.npr.org/podcasts/510333/throughline',
 'https://www.npr.org/series/770565791/npr-music-live-sessions',
 'https://www.npr.org/podcasts/510310/npr-politics-podcast',
 'https://www.npr.org/sections/allsongs/',
 'https://www.npr.org/podcasts/510311/embedded',
 'https://www.npr.org/podcasts/510374/trumps-terms',
 'https://www.npr.org/sections/music-features',
 'https://www.npr.org/series/tiny-desk-concerts/',
 'https://www.npr.org/sections/allsongs/606254804/new-music-friday',
 'https://www.npr.org/podcasts/510333/throughline',
 'https://www.npr.org/series/770565791/npr-music-live-sessions',
 'https://www.npr.org/podcasts/510310/npr-politics-podcast',
 'https://www.npr.org/sections/allsongs/',
 'https://www.npr.org/podcasts/510311/embedded',
 'https://www.npr.